In [10]:
import cv2
import os
from matplotlib import pyplot as plt
from PIL import ImageFont, ImageDraw, Image, ImageColor
import numpy as np
import glob
import xmltodict
from tqdm import tqdm
from psbody.mesh import Mesh

In [3]:
chokepoint_root = "/home/oole/Data/Chokepoint"
def _load_roidb_chokepoint(subset):
    subset_path = os.path.join(chokepoint_root, "annotation", "G1", subset)
    paths = sorted(glob.glob(subset_path + "*/*/*/*"))
    vid_names_1 = [path for path in paths if "xml" not in path and "seq" in path]
    vid_names_1 = ["/".join(v.split("/")[-3:]) for v in vid_names_1]
    paths = sorted(glob.glob(subset_path + "*/*/*"))
    vid_names_2 = [path for path in paths if "xml" not in path and "seq" in path]
    vid_names_2 = ["/".join(v.split("/")[-2:]) for v in vid_names_2]
    vid_names = list(vid_names_2 + vid_names_1)
    return vid_names

def get_sequences(subset):
    subset_path = os.path.join(chokepoint_root, "annotation", "G1", subset)
    paths = sorted(glob.glob(subset_path + "*/*/*/*"))
    sequences_1 = [path for path in paths if "xml" not in path and "seq" in path]
    sequences_1 = ["/".join(v.split("/")[-3:]) for v in sequences_1]
    paths = sorted(glob.glob(subset_path + "*/*/*"))
    sequences_2 = [path for path in paths if "xml" not in path and "seq" in path]
    sequences_2 = ["/".join(v.split("/")[-2:]) for v in sequences_2]
    sequences = list(sequences_2 + sequences_1)
    return sequences

In [4]:
def get_sequence_frame_files(sequence, subset):
    sequence_folder = os.path.join(chokepoint_root, "annotation/G1/", subset, sequence)
    sequence_frame_files = sorted(glob.glob(sequence_folder + "/*.xml"))
    sequence_frame_numbers = [frame_file.split("/")[-1].split(".")[0] for frame_file in sequence_frame_files]
    first_frame_number = sequence_frame_numbers[0]
    return sequence_frame_files, first_frame_number

In [5]:
# Some helper functions to draw image with object boundary boxes
fontname = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
font = ImageFont.truetype(fontname, 40) if os.path.isfile(fontname) else ImageFont.load_default()

def bounding_box(img, xmin, ymin, xmax, ymax, width, score):
    draw = ImageDraw.Draw(img)
    xres, yres = img.size[0], img.size[1]
    box = [xmin, ymin, xmax, ymax]
    draw.rectangle(box, outline="red", width=width)

def plot_img(img, axes, xmin, ymin, xmax, ymax):
    for i in range(len(xmin)):
        bounding_box(img, xmin[i], ymin[i], xmax[i], ymax[i], 2, -1)
    plt.setp(axes, xticks=[], yticks=[])
    plt.imshow(img)


In [6]:
def obj_data_to_bbox(annotation):
    object_ann = annotation['object']
    bbox = object_ann['bndbox']
    x1 = int(bbox['xmin'])
    y1 = int(bbox['ymin'])
    x2 = int(bbox['xmax'])
    y2 = int(bbox['ymax'])
    box = [x1, y1, x2, y2]
    return box

def obj_file_path(annotation):
    sub_path = annotation['folder']
    fname = annotation['filename']
    full_path = os.path.join(chokepoint_root, sub_path, fname + ".jpg")
    return full_path

In [7]:
# Ringnet preprocess:
def resize_img(img, scale_factor):
    new_size = (np.floor(np.array(img.shape[0:2]) * scale_factor)).astype(int)
    new_img = cv2.resize(img, (new_size[1], new_size[0]))
    # This is scale factor of [height, width] i.e. [y, x]
    actual_factor = [
        new_size[0] / float(img.shape[0]), new_size[1] / float(img.shape[1])
    ]
    return new_img, actual_factor

def scale_and_crop(image, scale, center, img_size):
    image_scaled, scale_factors = resize_img(image, scale)
    # Swap so it's [x, y]
    scale_factors = [scale_factors[1], scale_factors[0]]
    center_scaled = np.round(center * scale_factors).astype(np.int)

    margin = int(img_size / 2)
    image_pad = np.pad(
        image_scaled, ((margin, ), (margin, ), (0, )), mode='edge')
    center_pad = center_scaled + margin
    # figure out starting point
    start_pt = center_pad - margin
    end_pt = center_pad + margin
    # crop:
    crop = image_pad[start_pt[1]:end_pt[1], start_pt[0]:end_pt[0], :]
    proc_param = {
        'scale': scale,
        'start_pt': start_pt,
        'end_pt': end_pt,
        'img_size': img_size
    }

    return crop, proc_param

def preprocess_image(img_path):
    img = io.imread(img_path)
    if np.max(img.shape[:2]) != 224:
        print('Resizing so the max image size is %d..' % 224)
        scale = (float(224) / np.max(img.shape[:2]))
    else:
        scale = 1.0#scaling_factor
    center = np.round(np.array(img.shape[:2]) / 2).astype(int)
    # image center in (x,y)
    center = center[::-1]
    crop, proc_param = scale_and_crop(img, scale, center,
                                               224)
    # import ipdb; ipdb.set_trace()
    # Normalize image to [-1, 1]
    # plt.imshow(crop/255.0)
    # plt.show()
    crop = 2 * ((crop / 255.) - 0.5)

    return crop, proc_param, img

In [9]:
# Create and save meshes using ringnet:
import tensorflow._api.v2.compat.v1 as tf
from psbody.mesh import Mesh
from skimage import io
import time
import pickle


face_data_path = os.path.join(chokepoint_root, "face_data")
first_batch_img_paths = sorted(glob.glob(face_data_path + "*/*/*/*/*/*"))
first_batch_img_paths = [path for path in first_batch_img_paths if ".jpg" in path]
# only jpgs
second_batch_img_paths = sorted(glob.glob(face_data_path + "*/*/*/*/*/*/*"))
second_batch_img_paths = [path for path in second_batch_img_paths if ".jpg" in path]

all_img_paths = first_batch_img_paths + second_batch_img_paths
assert len(all_img_paths) == len(np.unique(all_img_paths)), "inconsistency with image paths! {} != {}".format(len(all_img_paths), en(np.unique(all_img_paths)))
# print(len(all_img_paths))
# print(len(np.unique(all_img_paths)))

template_mesh = Mesh(filename="/home/oole/git/ma/coma/impl/data/template.obj")
# viewer = MeshViewer()
load_path = "/mnt/storage/Msc/RingNet/model/ring_6_68641"
model_path = load_path + ".meta"

chkpoint_flame_parameters_root = "/mnt/storage/Data/Chokepoint"
flame_path = os.path.join(chkpoint_flame_parameters_root, "face_data_flame_parameters")
# print("flame_path: {}".format(flame_path))

flame_parameters_list = []

with tf.Session() as sess:
    graph = sess.graph
    saver = tf.compat.v1.train.import_meta_graph(model_path)
    saver.restore(sess, load_path)
    vertices = graph.get_tensor_by_name(u'Flamenetnormal_2/Add_9:0')
    params = graph.get_tensor_by_name(u'add_2:0')
    image_input = graph.get_tensor_by_name(u'input_images:0')
    
    i = 0
    print("start processing")
    for img_path in tqdm(all_img_paths):
#         sub_path_to_mesh = img_path.split(chokepoint_root)[-1].split("/face_data")[-1].split(".jpg")[0][1:]
#         sub_path_split = sub_path_to_mesh.split("/")
#         fname = sub_path_split[-1] + ".pickle"
#         sub_path = ("/").join(sub_path_split[:-1])
#         full_path =os.path.join(flame_path, sub_path)
#         print(full_path)
#         if not os.path.exists(full_path):
#             os.makedirs(full_path)
#         full_path_to_flame = os.path.join(full_path, fname)
#         full_path_fo_flame = full_path_to_flame.replace(chokepoint_root, chkpoint_flame_parameters_root)
#         print(full_path_fo_flame)
#         print(full_path_to_flame)
        crop, proc_param, img = preprocess_image(img_path)
        face_image = np.expand_dims(crop, axis=0)
        # input_image = np.expand_dims(face_image, axis=0)
        fetch_dict = {'vertices': vertices,
                     'parameters': params}
        feed_dict = {
                image_input: face_image
            }
        res = sess.run(fetch_dict, feed_dict)
        verts = res['vertices'][0]
        parameters = res['parameters'][0]
#         flame_parameters = res['parameters']
#         viewer.set_dynamic_meshes([Mesh(v=verts, f=template_mesh.f)])
#         print(full_path_to_mesh)
        
        flame_parameters = {
            'img_path': img_path,
#             'crop': crop,
            'proc_param': proc_param,
#             'img': img,
            'verts': verts,
            'parameters': parameters
        }
        flame_parameters_list.append(flame_parameters)

        
#         time.sleep(5)


INFO:tensorflow:Restoring parameters from /mnt/storage/Msc/RingNet/model/ring_6_68641


  0%|          | 0/37315 [00:00<?, ?it/s]

start processing


100%|██████████| 37315/37315 [05:27<00:00, 113.97it/s]


In [10]:
pickle.dump(flame_parameters_list, open(os.path.join("/home/oole/Data/Chokepoint/face_data_flame_parameters", "flame_parameters.pickle"), 'wb'), protocol=2)

In [13]:
# Create and save meshes using ringnet:
import tensorflow._api.v2.compat.v1 as tf
from psbody.mesh import Mesh
from skimage import io
import time
import pickle


template_mesh = Mesh(filename="/home/oole/git/ma/coma/impl/data/template.obj")
# viewer = MeshViewer()
load_path = "/mnt/storage/Msc/RingNet/model/ring_6_68641"
model_path = load_path + ".meta"


my_image = "/home/oole/Desktop/oole.jpg"
with tf.Session() as sess:
    graph = sess.graph
    saver = tf.compat.v1.train.import_meta_graph(model_path)
    saver.restore(sess, load_path)
    vertices = graph.get_tensor_by_name(u'Flamenetnormal_2/Add_9:0')
    params = graph.get_tensor_by_name(u'add_2:0')
    image_input = graph.get_tensor_by_name(u'input_images:0')
    
    
    crop, proc_param, img = preprocess_image(my_image)
    face_image = np.expand_dims(crop, axis=0)
        
    fetch_dict = {'vertices': vertices,
                 'parameters': params}
    feed_dict = {
            image_input: face_image
        }
    res = sess.run(fetch_dict, feed_dict)
    verts = res['vertices'][0]
    parameters = res['parameters'][0]

    flame_parameters = {
        'img_path': my_image,
#             'crop': crop,
        'proc_param': proc_param,
#             'img': img,
        'verts': verts,
        'parameters': parameters
    }


INFO:tensorflow:Restoring parameters from /mnt/storage/Msc/RingNet/model/ring_6_68641


In [14]:
pickle.dump(flame_parameters, open(os.path.join("/home/oole/Desktop/", "oole_flame_parameters.pickle"), 'wb'), protocol=2)
